<a href="https://colab.research.google.com/github/deekshahs1817/Infosys_SpringBoard_All_milestones-1-4-/blob/main/dee_milestone_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit pyngrok easyocr pytesseract Pillow pyjwt
!pip install pdf2image



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 21.3 MB/s eta 0:00:00


In [ ]:
!ngrok config add-authtoken 36Qu9bgc0UlJm9PIWRmYG63fVVd_4BrsHqQFcornADrZcgR1t


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
%%writefile app.py
import streamlit as st
import pytesseract
from PIL import Image
import pandas as pd
import io, jwt, datetime
import re

# =====================
#   SIMPLE JWT SETUP
# =====================
SECRET_KEY = "internshipdemo"

def create_token(username):
    payload = {"user": username, "exp": datetime.datetime.utcnow() + datetime.timedelta(hours=1)}
    return jwt.encode(payload, SECRET_KEY, algorithm="HS256")

def verify_token(token):
    try:
        jwt.decode(token, SECRET_KEY, algorithms=["HS256"])
        return True
    except Exception:
        return False

# =====================
#   Aadhaar Details Extraction
# =====================
def extract_aadhaar_details(text):
    details = {"Name": "", "DOB": "", "AadhaarNumber": "", "Phone": "", "Address": ""}
    lines = [line.strip() for line in text.split("\n") if line.strip()]

    dob_index = -1
    aadhaar_index = -1
    phone_index = -1

    # Extract Aadhaar number, DOB, Phone
    for i, line in enumerate(lines):
        if not details["AadhaarNumber"]:
            match_aadhaar = re.search(r"\d{4}\s\d{4}\s\d{4}", line)
            if match_aadhaar:
                details["AadhaarNumber"] = match_aadhaar.group()
                aadhaar_index = i

        if not details["DOB"]:
            match_dob = re.search(r"\d{2}/\d{2}/\d{4}", line)
            if match_dob:
                details["DOB"] = match_dob.group()
                dob_index = i

        if not details["Phone"]:
            match_phone = re.search(r"\d{10}", line)
            if match_phone:
                details["Phone"] = match_phone.group()
                phone_index = i

    # Extract Name: first mostly alphabetic line above DOB
    if dob_index != -1:
        for i in range(dob_index - 1, -1, -1):
            candidate = lines[i]
            words = candidate.split()
            alpha_count = sum(c.isalpha() for c in candidate)
            if len(words) >= 2 and alpha_count / len(candidate.replace(" ", "")) > 0.8:
                # Remove single-letter prefix (OCR artifact)
                if len(words[0]) == 1 and words[0].isalpha():
                    words = words[1:]
                details["Name"] = " ".join(words).title()
                break

    # Extract Address: lines above phone number (3–6 lines)
    if phone_index != -1:
        address_lines = []
        count = 0
        for i in range(phone_index - 1, -1, -1):
            line = lines[i]
            if any(c.isdigit() for c in line) and len(line) <= 12:
                # Likely DOB or Aadhaar, stop
                break
            if line.strip():  # ignore empty lines
                address_lines.insert(0, line)
                count += 1
            if count >= 6:  # maximum 6 lines
                break
        details["Address"] = ", ".join(address_lines)

    return details

# =====================
#   STREAMLIT UI CONFIG
# =====================
st.set_page_config(page_title="Aadhaar OCR KYC", page_icon="🆔", layout="wide")
st.markdown("""
    <style>
        .stButton>button {background-color: #4CAF50; color: white; font-size:16px; height:40px;}
        .stTextInput>div>input, .stTextArea>div>textarea {font-size:16px; padding:6px;}
        .stDataFrame {font-size:14px;}
        .stFileUploader>div>input {font-size:16px;}
    </style>
""", unsafe_allow_html=True)

st.title("🆔 Aadhaar / KYC Document Upload System")
st.markdown("Upload your Aadhaar/PAN card to extract details via OCR.")

# =====================
#   LOGIN / AUTH
# =====================
if "token" not in st.session_state:
    st.subheader("🔑 Login")
    username = st.text_input("Enter username")
    if st.button("Login"):
        if username.strip() != "":
            st.session_state.token = create_token(username)
            st.success(f"Welcome {username}! You are logged in.")
        else:
            st.error("Username cannot be empty.")

# =====================
#   MAIN UI AFTER LOGIN
# =====================
if "token" in st.session_state and verify_token(st.session_state.token):

    st.subheader("📂 Upload Aadhaar / PAN Document")
    uploaded_file = st.file_uploader(
        "Choose a file (Image: jpg/png/jpeg or CSV)",
        type=["jpg", "jpeg", "png", "csv"]
    )

    if uploaded_file:
        # File size validation
        file_size = uploaded_file.size / (1024*1024)
        if file_size > 2:
            st.error("File size exceeds 2MB. Please upload a smaller file.")
        else:
            # ---- CSV file ----
            if uploaded_file.name.endswith(".csv"):
                try:
                    df = pd.read_csv(uploaded_file)
                    st.success("✅ CSV file loaded successfully")
                    st.subheader("📊 CSV Preview")
                    st.dataframe(df)
                except Exception as e:
                    st.error(f"Error reading CSV file: {e}")

            # ---- Image file ----
            elif uploaded_file.name.lower().endswith(("jpg", "jpeg", "png")):
                try:
                    image = Image.open(uploaded_file)
                    st.image(image, caption="Uploaded Document", use_container_width=True)

                    with st.spinner("Extracting Aadhaar details..."):
                        text = pytesseract.image_to_string(image)

                    st.success("✅ OCR Extraction Successful")
                    st.subheader("Extracted Details")
                    details = extract_aadhaar_details(text)

                    # Show extracted fields in separate boxes
                    st.text_input("Name", details["Name"])
                    st.text_input("Date of Birth", details["DOB"])
                    st.text_input("Aadhaar Number", details["AadhaarNumber"])
                    st.text_input("Phone Number", details["Phone"])
                    st.text_area("Address", details["Address"], height=100)

                    if st.button("Confirm & Save"):
                        st.success("Details saved successfully ✅")

                except Exception as e:
                    st.error(f"Error processing the file: {e}")

            else:
                st.error("Invalid file type. Please upload an image (jpg/png) or CSV file.")

    st.markdown("---")
    if st.button("Logout"):
        st.session_state.pop("token")
        st.success("Logged out successfully.")

else:
    st.info("Please login to use the upload feature.")


Writing app.py


In [ ]:
!curl -s http://localhost:4040/api/tunnels | jq -r '.tunnels[0].public_url'


In [ ]:
!curl ipv4.icanhazip.com

35.194.244.67


In [ ]:
!streamlit run app.py --server.port 8501 --server.address 0.0.0.0 &>/dev/null &

In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("36Qu9bgc0UlJm9PIWRmYG63fVVd_4BrsHqQFcornADrZcgR1t")  # put your token here
public_url = ngrok.connect(8501)
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://9e876ac992f7.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!ngrok version

ngrok version 3.34.0
pyngrok version 7.5.0
